In [8]:
from tensorflow.python.summary import summary_iterator

#summary_reader = tbx.SummaryReader("C:/Users/vchad/Desktop/scout_double_opt_ray_results/")
#tag_data = summary_reader.#get_data("scalar")
#print(tag_data)

ImportError: cannot import name 'text_format' from 'google.protobuf' (C:\Users\vchad\AppData\Roaming\Python\Python39\site-packages\google\protobuf\__init__.py)

In [2]:
import os
import numpy as np
import pandas as pd
from scipy.stats import ttest_ind, mannwhitneyu

osl = os.listdir
ospj = os.path.join

### get relevant runs data given filters of interest

In [3]:
import re

def natural_sort(l): 
    convert = lambda text: int(text) if text.isdigit() else text.lower() 
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)', key)] 
    return sorted(l, key=alphanum_key)

In [16]:
"""
runs_dir = "/home/derposoft/ray_results"
runs_filters = [
    "fcscout_baseline200h_1r2b_SEED",
    "gnnscout_200h2.2l_1r2b_SEED",
    "gatscout_200h2.2l_1r2b_SEED"
]
runs_filters = [
    "fcskirmish_5hp_baseline100_50_PAPER_SEED",
    "gcnskirmish_5hp_baseline100_50_3.4_PAPER_SEED",
    "gatLNskirmish_5hp_GLOBALbaseline100_50_3.4_PAPER_SEED",
    "gcnLNskirmish_5hp_baseline100_50_3.4_PAPER_SEED",
]
runs_filters = [
    "fcskirmish_5hp_baseline100_50_100ksteps_SEED",
    "gcnskirmish_5hp_baseline100_50_2.4_PAPER_SEED",
]
runs_dirs = [
    "/home/derposoft/Documents/tb_logs/40k_runs/fcskirmish_100_50",
    "/home/derposoft/Documents/tb_logs/40k_runs/gcnskirmish_100_50_3.4",
]
runs_filters = [
    "fcskirmish_5hp_baseline100_50_PAPER_SEED",
    "gcnskirmish_5hp_baseline100_50_3.4_PAPER_SEED",
]
"""
"""
runs_dirs = [
    "/home/derposoft/Documents/tb_logs/seeded_runs/",
    "/home/derposoft/Documents/tb_logs/tb_logs.2022-12-18_paper/"
]
runs_filters_5hp = [
    "baseline_SEED",
    "hybridGAT_5hp_globalemb_NOOPT_SEED",
    "hybridGAT_5hp_hybridemb_NOOPT_SEED",
    "hybridGAT_5hp_localemb_NOOPT_SEED",
    "hybridGCN_5hp_globalemb_NOOPT_SEED",
    "hybridGCN_5hp_hybridemb_NOOPT_SEED",
    "hybridGCN_5hp_localemb_NOOPT_SEED",
    "hybridGT_5hp_globalemb_NOOPT_SEED",
    "hybridGT_5hp_hybridemb_NOOPT_SEED",
    "hybridGT_5hp_localemb_NOOPT_SEED",
]
runs_filters_2hp = [
    "baseline_2hp_NOOPT_SEED",
    "hybridGAT_2hp_localemb_NOOPT_SEED",
    "hybridGCN_2hp_localemb_NOOPT_SEED",
    "hybridGT_2hp_localemb_NOOPT_SEED",
]
runs_filters_20hp = [
    "baseline_20hp_NOOPT_SEED",
    "hybridGAT_20hp_localemb_NOOPT_SEED",
    "hybridGCN_20hp_localemb_NOOPT_SEED",
    "hybridGT_20hp_localemb_NOOPT_SEED",
]
runs_filters_5hp_2v2 = [
    "baseline_5hp_NOOPT_2r2b_SEED",
    "hybridGAT_5hp_localemb_NOOPT_2r2b_SEED",
    "hybridGCN_5hp_localemb_NOOPT_2r2b_SEED",
    "hybridGT_5hp_localemb_NOOPT_2r2b_SEED",
]
"""
runs_dirs = ["/home/vchad/ray_results/"]
runs_filters_scout_2v2 = [
    "fcscout_200h2.2l_2r2b",
    "gcnscout_200h2.2l_2r2b",
    "gatscout_200h2.2l_2r2b",
]
runs_filters_scout_2v2_OPT = [
    "fcscout_200h2.2l_2r2b_OPT",
    "gcnscout_200h2.2l_2r2b_OPT",
    "gatscout_200h2.2l_2r2b_OPT",
]
runs_filters = runs_filters_scout_2v2_OPT
runs = sum(([ospj(runs_dir, x) for x in osl(runs_dir)] for runs_dir in runs_dirs), [])
tot_steps_len = 200
runs = natural_sort(runs)

In [17]:
all_filters_runs = [
    [run for run in runs if runs_filter in run]
    for runs_filter in runs_filters
]

In [18]:
run_data_file = "progress.csv"
run_data_col = "episode_reward_mean"
# run_data_col = "evaluation/episode_reward_mean"
# run_data_col = "ray/tune/episode_reward_mean"
run_step_col = "timesteps_total"
all_runs_data = []
for filter_runs in all_filters_runs:
    curr_run_data = []
    for run in filter_runs:
        datafile = pd.read_csv(ospj(run, run_data_file))
        curr_run_data.append(datafile[run_data_col])
    all_runs_data.append(curr_run_data)
runs_steps = [
    pd.read_csv(ospj(run, run_data_file))[run_step_col]
    for run in runs
]

### given data, run "map" on data to get values suitable for analysis

In [19]:
# finds the first training step when reward exceeds a given value ge
def first_ep_to_val(runs, steps, ge=30):
    first_eps = []
    for run, steps in zip(runs, steps):
        found = False
        for step, val in zip(steps, run):
            if val >= ge:
                first_eps.append(step)
                found = True
                break
        if not found:
            first_eps.append(-1)
    return first_eps

# finds max avg reward
def max_avg_reward(runs, steps):
    max_vals = []
    for run in runs:
        if len(run) < tot_steps_len: continue
        max_vals.append(max(run))
    return max_vals

# returns last reward
def last_ep_reward(runs, steps):
    last_vals = []
    for run in runs:
        if len(run) < tot_steps_len: continue
        last_vals.append(run.tolist()[-1])
    return last_vals

In [20]:
first_ep_to_30 = [
    first_ep_to_val(filter_runs_data, runs_steps)
    for filter_runs_data in all_runs_data
]
max_reward = [
    max_avg_reward(filter_runs_data, runs_steps)
    for filter_runs_data in all_runs_data
]
last_reward = [
    last_ep_reward(filter_runs_data, runs_steps)
    for filter_runs_data in all_runs_data
]

### given "map" values, run "reduce" to aggregate values and show significance

In [21]:
# pretty print a dictionary
def pretty_print(d, tabs=0, tabsize=4):
    for k in d:
        v = d[k]
        nspaces = " " * tabsize * tabs
        if type(v) == dict:
            print(f"{nspaces}{k}:")
            pretty_print(v, tabs+1, tabsize)
        else:
            print(f"{nspaces}{k}: {v}")

# get stats for mapped values for a given experiment
def get_stats(all_runs_vals, firstn=-1):
    stats = {}
    #baselines = all_runs_vals[0]
    baselines = all_runs_vals[0]
    for filter, vals in zip(runs_filters, all_runs_vals):
        if firstn > 0: vals = vals[:firstn]
        mu = np.mean(vals)
        sigma = np.std(vals)
        ci95z = 1.96
        ttest_results = ttest_ind(baselines, vals, alternative="less", equal_var=False)
        stats[filter] = {
            "str": f"{mu}+/-{ci95z*sigma} -- p={ttest_results.pvalue}",
            #"mean": f"{mu}+/-{ci95z*sigma}",
            #"median": np.median(vals),
            #"std": sigma,
            #"ci": [mu-ci95z*sigma, mu+ci95z*sigma],
            #"n": len(vals),
            #"mannwu": mannwhitneyu(baselines, vals, alternative="less"),
            #"ttest_less": ttest_ind(baselines, vals, alternative="less", equal_var=False),
            #"ttest_more": ttest_ind(baselines, vals, alternative="greater", equal_var=False),
        }
    return stats

In [22]:
firstn = -1
first_ep_to_30_stats = get_stats(first_ep_to_30, firstn=firstn)
max_reward_stats = get_stats(max_reward, firstn=firstn)
last_reward_stats = get_stats(last_reward, firstn=firstn)

/tmp/ipykernel_3677444/2837841425.py:22: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  ttest_results = ttest_ind(baselines, vals, alternative="less", equal_var=False)


In [23]:
def print_stat(stats, name=""):
    assert name != ""
    print(name+" stats:")
    pretty_print(stats)
    print()

print_stat(max_reward_stats, "max reward")
print_stat(first_ep_to_30_stats, "first ep to 30")
print_stat(last_reward_stats, "last reward")


max reward stats:
fcscout_200h2.2l_2r2b:
    str: 315.88399999999996+/-97.01067834653254 -- p=0.5
gcnscout_200h2.2l_2r2b:
    str: 291.36800000000005+/-81.03147943633756 -- p=0.8652437992802904
gatscout_200h2.2l_2r2b:
    str: 308.45400000000006+/-84.40203387145594 -- p=0.6309763643621769

first ep to 30 stats:
fcscout_200h2.2l_2r2b:
    str: 400.0+/-0.0 -- p=nan
gcnscout_200h2.2l_2r2b:
    str: 400.0+/-0.0 -- p=nan
gatscout_200h2.2l_2r2b:
    str: 400.0+/-0.0 -- p=nan

last reward stats:
fcscout_200h2.2l_2r2b:
    str: 310.965+/-87.87791408853536 -- p=0.5
gcnscout_200h2.2l_2r2b:
    str: 289.265+/-79.53091652216766 -- p=0.8519812674069078
gatscout_200h2.2l_2r2b:
    str: 305.60800000000006+/-79.4189696561319 -- p=0.6033281625887179

